<p> Imports </p>

In [1]:
# %pip install spotipy
# %pip install python-levenshtein
# %pip install fuzzywuzzy
import pandas as pd
import numpy as np
import spotipy.oauth2 as oauth2
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.preprocessing import MultiLabelBinarizer
import time
import fuzzywuzzy

<p> Spotipy Credentials </p>

In [2]:
auth_manager = SpotifyClientCredentials(client_id = '7606efb22a6848bdbf48ecd67484b9d5', 
                                        client_secret = '5febce52b67b499db9f7295c5f6839ec')
sp = spotipy.Spotify(auth_manager=auth_manager)

<p> The following cell loads in the grammy award winning and selects certain columns. It also drops duplicates and removes hyphens. </p>

In [3]:
# loadin and group grammy awards
grammyAwards = pd.read_csv("data/grammySongs_1999-2019.csv")
grammyAwards = grammyAwards[['Name', 'Artist', 'GrammyAward', 'GrammyYear', 'Genre']]
grammyAwards = grammyAwards.groupby(['Name', 'Artist', 'GrammyYear', 'Genre']).count().reset_index()
grammyAwards = grammyAwards.drop_duplicates(subset=['Name', 'Artist'], keep='last').reset_index(drop=True)
grammyAwards['Artist'] = grammyAwards['Artist'].apply(lambda s: s.replace("-", " "))
grammyAwards = grammyAwards.applymap(lambda s: s.upper() if type(s) == str else s)

grammyAwards.head(5)

,Name,Artist,GrammyYear,Genre,GrammyAward
0,'TIL SUMMER COMES AROUND,KEITH URBAN,2010,COUNTRY,1
1,24 FRAMES,"ASON ISBELL, SONGWRITER",2015,AMERICAN ROOTS MUSIC,1
2,24K MAGIC,BRUNO MARS,2017,GENERAL,1
3,500 MILES HIGH,"CHICK COREA, SOLOIST",2011,JAZZ,1
4,99 PROBLEMS,JAY Z,2004,RAP,1


<p> The following code fixes the artist names by querying the spotipy API, or by performing some string splitting. </p>

In [4]:
for index, row in grammyAwards.iterrows():
    artistName = row['Artist']
    trackName = row['Name']
    year = row['GrammyYear']
    try: 
        artistList = artistName.split(",")
        artistName = artistList[0]
        artistName = artistName.split("&")[0]
        artistName = artistName.split(":")[0]
        artistName = artistName.split(";")[0]
        artistName = artistName.split(":")[0]
        artistName = artistName.replace("-", " ")
        if 'FEATURING' in artistName:
            grammyAwards.at[index, 'Artist'] = artistName.split('FEATURING')[0]
            continue
        if 'SOLOIST' in row['Artist']:
            grammyAwards.at[index, 'Artist'] = artistName.split(",")[0]
            continue
        if (artistName != row['Artist']):
            song = sp.search(q=f"track:{trackName}, year:{year}", type="track", limit=2)
            if len(song['tracks']['items']) == 0:
                song = sp.search(q=f"track:{trackName}", type="track", limit=2)
            artist = song['tracks']['items'][0]['artists'][0]['name'].upper()
            grammyAwards.at[index, 'Artist'] = artist
    except:
        print("track not found", trackName)
grammyAwards.head(5)

track not found A FEATHER'S NOT A BIRD


,Name,Artist,GrammyYear,Genre,GrammyAward
0,'TIL SUMMER COMES AROUND,KEITH URBAN,2010,COUNTRY,1
1,24 FRAMES,JASON ISBELL,2015,AMERICAN ROOTS MUSIC,1
2,24K MAGIC,BRUNO MARS,2017,GENERAL,1
3,500 MILES HIGH,CHICK COREA,2011,JAZZ,1
4,99 PROBLEMS,JAY Z,2004,RAP,1


<p> The following code loads in and cleans the billboard data, fills in null values, finds the maximum weeks on chart, top position, average weekly rank. </p>

In [ ]:
# loading and cleaning billboard
billboardHot100 = pd.read_csv('data/billboardHot100_1999-2019.csv')
billboardHot100 = billboardHot100[['Artists', 'Name', 'Genre', 'Peak.position', 'Weeks.on.chart','Weekly.rank','Date']]
billboardHot100['Weeks.on.chart'] = billboardHot100['Weeks.on.chart'].fillna(1)
billboardHot100['Peak.position'] = billboardHot100['Peak.position'].fillna(100)
billboardHot100['Weekly.rank'] = billboardHot100['Weekly.rank'].fillna(100)
billboardHot100['Genre'] = billboardHot100['Genre'].fillna('None')
billboardHot100['Date'] = billboardHot100['Date'].fillna('1999')
billboardHot100 = billboardHot100.groupby(['Artists', 'Name','Genre']).agg({'Peak.position':'min', 'Weeks.on.chart':'max', 'Weekly.rank':'mean', 'Date':'max'})[['Peak.position','Weeks.on.chart','Weekly.rank', 'Date']].reset_index()
billboardHot100.rename(columns = {'Weekly.rank':'Avg.rank', 'Artists':'Artist'}, inplace = True)
billboardHot100['Date'] = pd.to_datetime(billboardHot100['Date'])
billboardHot100['Date'] = billboardHot100['Date'].apply(lambda x : x.year)
billboardHot100 = billboardHot100.applymap(lambda s: s.upper() if type(s) == str else s)
billboardHot100.head()

<p> The following code gets song statistics for billboard songs through the spotipy api, saves it in a CSV</p>

In [ ]:
for index, row in billboardHot100.iterrows():
    artistName = row['Artist']
    trackName = row['Name']
    year = row['Date']
    try: 
        song = sp.search(q=f"track:{trackName}, year:{year}", type="track", limit=2)
        if len(song['tracks']['items']) == 0:
            song = sp.search(q=f"track:{trackName}", type="track", limit=2)
        track_id = song['tracks']['items'][0]['uri']
        song_info = sp.audio_features(track_id)           
        billboardHot100.at[index, 'Energy'] = song_info[0].get('energy')
        billboardHot100.at[index, 'Danceability'] = song_info[0].get('danceability')
        billboardHot100.at[index, 'Tempo'] = song_info[0].get('tempo')
        billboardHot100.at[index, 'Valence'] = song_info[0].get('valence')
    except:
        print("track not found", trackName)
    if (index % 500 == 0):
        print(index)
# billboardHot100.to_csv("billboard_features.csv")

<p> Load in the billboard csv, because the above cell does a large number of queries that we don't want to repeat. It also creates a column notating if the billboard song won a grammy or not.</p>

In [5]:
billboardHot100 = pd.read_csv('billboard_features.csv')
billboardHot100['GrammyAward'] = np.zeros((len(billboardHot100)), dtype=int)
billboardHot100.head(5)

,Unnamed: 0,Artist,Name,Genre,Peak.position,Weeks.on.chart,Avg.rank,Date,Energy,Danceability,Tempo,Valence,GrammyAward
0,0,"""WEIRD AL"" YANKOVIC",CANADIAN IDIOT,"COMEDY,PARODY,ROCK",82.0,3.0,91.333333,2006,0.697,0.543,185.978,0.861,0
1,1,"""WEIRD AL"" YANKOVIC",WORD CRIMES,"PARODY,COMEDY,POP",39.0,2.0,65.000000,2014,0.430,0.897,121.987,0.964,0
2,2,'N SYNC,BYE BYE BYE,"ADULT CONTEMPORARY,DANCE-POP,TEEN POP,BOY BAND...",4.0,23.0,14.260870,2000,0.926,0.610,172.638,0.861,0
3,3,'N SYNC,GONE,"ADULT CONTEMPORARY,ACOUSTIC,SOUL POP,R&;B,BOY ...",11.0,24.0,27.125000,2001,0.409,0.704,113.863,0.495,0
4,4,'N SYNC,IT'S GONNA BE ME,"MEMES,DANCE-POP,TEEN POP,BOY BAND,POP",1.0,25.0,21.680000,2000,0.873,0.636,165.071,0.908,0


<p> The following code merges the grammyAwards table and the billboardHot100 tables with a fuzzy merge. The naming convention for the artists is slightly different between the two tables, so we are forced to use levenstein distance to compute similar rows. This is a slow process and requires a lot of computation, but we could not figure out a better way. Saves results in a csv. This function also updates the artists of the grammy csv to match the billboard. Also, this sets grammy winners as winners in the billboard csv.</p>

In [6]:
# code for merging

'''
for each one, search in the billboard for the artistName, true if satisfies fuzzy and 
'''
from fuzzywuzzy import fuzz
for index, row in grammyAwards.iterrows():
    if index % 50 == 0:
        print(index)
    artist1 = row['Artist']
    track = row['Name']
    found = False
    for id2, r2 in billboardHot100.iterrows():
        artist2 = r2['Artist']
        t2 = r2['Name']
        artistRatio = fuzz.partial_ratio(artist1, artist2)
        trackRatio = fuzz.partial_ratio(track, t2)
        if trackRatio > 90 and artistRatio > 90:
            print(artist1, artist2)
            print(track, t2)
            found = True
            grammyAwards.at[index, 'Artist'] = artist2
            grammyAwards.at[index, 'Energy'] = r2['Energy']
            grammyAwards.at[index, 'Danceability'] = r2['Danceability']
            grammyAwards.at[index, 'Tempo'] = r2['Tempo']
            grammyAwards.at[index, 'Valence'] = r2['Valence']
            grammyAwards.at[index, 'Peak.position'] = r2['Peak.position']
            grammyAwards.at[index, 'Weeks.on.chart'] = r2['Weeks.on.chart']
            grammyAwards.at[index, 'Avg.rank'] = r2['Avg.rank']
            billboardHot100.at[id2, 'GrammyAward'] = 1
            break
    if not found:
        grammyAwards.at[index, 'Energy'] = -1
        grammyAwards.at[index, 'Danceability'] = -1
        grammyAwards.at[index, 'Tempo'] = -1
        grammyAwards.at[index, 'Valence'] = -1
        grammyAwards.at[index, 'Peak.position'] = -1
        grammyAwards.at[index, 'Weeks.on.chart'] = -1
        grammyAwards.at[index, 'Avg.rank'] = -1
        print('no match in top 100', track, artist1)
grammyAwards.head(10)
grammyAwards.to_csv("grammy_award_data2.csv")

0
no match in top 100 'TIL SUMMER COMES AROUND KEITH URBAN
no match in top 100 24 FRAMES JASON ISBELL
BRUNO MARS BRUNO MARS
24K MAGIC 24K MAGIC
no match in top 100 500 MILES HIGH CHICK COREA
JAY Z JAY
99 PROBLEMS 99 PROBLEMS
MIGUEL MIGUEL
ADORN ADORN
LENNY KRAVITZ LENNY KRAVITZ
AGAIN AGAIN
CHRISTINA AGUILERA CHRISTINA AGUILERA
AIN'T NO OTHER MAN AIN'T NO OTHER MAN
PARAMORE PARAMORE
AIN'T IT FUN AIN'T IT FUN
KENDRICK LAMAR KENDRICK LAMAR
ALRIGHT ALRIGHT
KENDRICK LAMAR KENDRICK LAMAR
ALRIGHT ALRIGHT
LENNY KRAVITZ LENNY KRAVITZ
AMERICAN WOMAN AMERICAN WOMAN
no match in top 100 ANGEL LALAH HATHAWAY
no match in top 100 ANGRY WORLD NEIL YOUNG
SYSTEM OF A DOWN SYSTEM OF A DOWN
B.Y.O.B. B.Y.O.B.
LADY GAGA LADY GAGA
BAD ROMANCE BAD ROMANCE
SKRILLEX  SKRILLEX
BANGARANG BANGARANG
no match in top 100 BARTON HOLLOW THE CIVIL WARS
no match in top 100 BE OK WILLI.AM
MARY J BLIGE MARY J. BLIGE
BE WITHOUT YOU BE WITHOUT YOU
CHRISTINA AGUILERA CHRISTINA AGUILERA
BEAUTIFUL BEAUTIFUL
U2 U2
BEAUTIFUL DAY B

<p> Load in the CSV containing grammy award winning songs with billboard data. </p>

In [ ]:
grammyAwards = pd.read_csv('grammy_award_data.csv')

<p> One hot encodes the genres in the billboard </p>

In [7]:
from sklearn.preprocessing import MultiLabelBinarizer

genres_as_list = [g.split(",") for g in billboardHot100['Genre']]
one_hot = MultiLabelBinarizer()
genre_encoding = one_hot.fit_transform(genres_as_list)
genres = one_hot.classes_
genres = [genre.replace(';', '') for genre in genres]

genre_df = pd.DataFrame(genre_encoding, columns = genres)
billboardHot100 = billboardHot100.join(genre_df)
billboardHot100 = billboardHot100.drop(columns=['Genre'])
billboardHot100

,Unnamed: 0,Artist,Name,Peak.position,Weeks.on.chart,Avg.rank,Date,Energy,Danceability,Tempo,...,TROPICAL HOUSE,TV,UK,UK RAP,UNDERGROUND HIP-HOP,WEST COAST,WORLD MUSIC,WORSHIP,YOUTUBE,\XD0\XA0\XD0\XBE\XD1\X81\XD1\X81\XD0\XB8\XD1\X8F (RUSSIA)
0,0,"""WEIRD AL"" YANKOVIC",CANADIAN IDIOT,82.0,3.0,91.333333,2006,0.697,0.543,185.978,...,0,0,0,0,0,0,0,0,0,0
1,1,"""WEIRD AL"" YANKOVIC",WORD CRIMES,39.0,2.0,65.000000,2014,0.430,0.897,121.987,...,0,0,0,0,0,0,0,0,0,0
2,2,'N SYNC,BYE BYE BYE,4.0,23.0,14.260870,2000,0.926,0.610,172.638,...,0,0,0,0,0,0,0,0,0,0
3,3,'N SYNC,GONE,11.0,24.0,27.125000,2001,0.409,0.704,113.863,...,0,0,0,0,0,0,0,0,0,0
4,4,'N SYNC,IT'S GONNA BE ME,1.0,25.0,21.680000,2000,0.873,0.636,165.071,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,7209,WILL.I.AM,FALL DOWN,100.0,1.0,58.000000,2013,0.902,0.538,159.925,...,0,0,0,0,0,0,0,0,0,0
7210,7210,WILL.I.AM,I GOT IT FROM MY MAMA,31.0,9.0,56.666667,2007,0.777,0.888,118.998,...,0,0,0,0,0,0,0,0,0,0
7211,7211,WILL.I.AM,IT'S A NEW DAY,78.0,2.0,81.000000,2008,0.644,0.759,119.804,...,0,0,0,0,0,0,0,0,0,0
7212,7212,"WILL.I.AM, BRITNEY SPEARS",SCREAM & SHOUT,3.0,24.0,14.583333,2012,0.686,0.797,127.990,...,0,0,0,0,0,0,0,0,0,0


<p> Finds the 50 most popular genres </p>

In [8]:
genre_count = []
genre_dict= {}
for genre in genres:
    count = billboardHot100[genre].sum()
    genre_count.append(count)
    genre_dict[genre] = count
sorted_genres = dict(sorted(genre_dict.items(), key=lambda genre:genre[1]))
popular_genres_top_50 = list(sorted_genres.keys())[-50:]

<p> Gets rid of all genres but the 50 most popular </p>

In [9]:
billboardHot100 = billboardHot100[list(billboardHot100.columns[1:12]) + popular_genres_top_50]

<p> Adds columns in the grammyAwards matrix for genres </p>

In [10]:
df = pd.DataFrame(data=np.zeros((360, len(popular_genres_top_50)), dtype=int), columns=popular_genres_top_50)
df.head()
grammyGenres = pd.concat([grammyAwards, df], axis=1)
grammyGenres.head(5)

,Name,Artist,GrammyYear,Genre,GrammyAward,Energy,Danceability,Tempo,Valence,Peak.position,...,CANADA,SOUNDTRACK,ALTERNATIVE ROCK,POP-ROCK,TRAP,COUNTRY,R&B,ROCK,RAP,POP
0,'TIL SUMMER COMES AROUND,KEITH URBAN,2010,COUNTRY,1,-1.000,-1.000,-1.000,-1.000,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,24 FRAMES,JASON ISBELL,2015,AMERICAN ROOTS MUSIC,1,-1.000,-1.000,-1.000,-1.000,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,24K MAGIC,BRUNO MARS,2017,GENERAL,1,0.803,0.818,106.970,0.632,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,500 MILES HIGH,CHICK COREA,2011,JAZZ,1,-1.000,-1.000,-1.000,-1.000,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,99 PROBLEMS,JAY,2004,RAP,1,0.887,0.494,89.554,0.548,30.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<p> Uses fuzzy to match each song with its related genres </p>

In [11]:
for index, row in grammyGenres.iterrows():
    givenGenre = row['Genre']
    for genre in popular_genres_top_50:
        genreRatio = fuzz.partial_ratio(givenGenre, genre)
        if (genreRatio > 90):
            grammyGenres.at[index, genre] = 1
grammyGenres = grammyGenres.drop(columns="Genre")
grammyGenres.head(5)

,Name,Artist,GrammyYear,GrammyAward,Energy,Danceability,Tempo,Valence,Peak.position,Weeks.on.chart,...,CANADA,SOUNDTRACK,ALTERNATIVE ROCK,POP-ROCK,TRAP,COUNTRY,R&B,ROCK,RAP,POP
0,'TIL SUMMER COMES AROUND,KEITH URBAN,2010,1,-1.000,-1.000,-1.000,-1.000,-1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,24 FRAMES,JASON ISBELL,2015,1,-1.000,-1.000,-1.000,-1.000,-1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,24K MAGIC,BRUNO MARS,2017,1,0.803,0.818,106.970,0.632,4.0,41.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,500 MILES HIGH,CHICK COREA,2011,1,-1.000,-1.000,-1.000,-1.000,-1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,99 PROBLEMS,JAY,2004,1,0.887,0.494,89.554,0.548,30.0,12.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


<p> Saves final CSVs </p>

In [12]:
grammyGenres = grammyGenres[['Name', 'Artist', 'GrammyAward', 'Avg.rank', 'Peak.position', 'Weeks.on.chart',
'Danceability', 'Energy', 'Tempo', 'Valence']+popular_genres_top_50]
grammyGenres.to_csv("Grammys.csv")
billboardHot100 = billboardHot100[['Name', 'Artist', 'GrammyAward', 'Avg.rank', 'Peak.position', 'Weeks.on.chart',
'Danceability', 'Energy', 'Tempo', 'Valence']+popular_genres_top_50]
billboardHot100.to_csv("Billboard.csv")